# Imports

In [2]:
import sys
sys.path.append('../..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from src.cleaning import prep_high_school_dataframe
from src.cps_model import print_cv_results

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Preprocessing via the SchoolYear Class

The preprocessing is handled by the SchoolYear class and its methods.

In [7]:
sy_1819_hs = prep_high_school_dataframe('../../data/chicago_data_portal_csv_files/Chicago_Public_Schools_-_School_Profile_Information_SY1819.csv',
                     '../../data/chicago_data_portal_csv_files/Chicago_Public_Schools_-_School_Progress_Reports_SY1819.csv')

0 Student Count
2 schools
15     ENGLEWOOD STEM HS
582       YCCS - VIRTUAL
Name: Short_Name_sp, dtype: object
All 0 Student Count Schools Dropped
0 Graduation Rate
2 schools
261    PATHWAYS - AVONDALE HS
343     NORTHSIDE LEARNING HS
Name: Short_Name_sp, dtype: object
##########
NA Graduation Rates
38 schools
All 0/NA Graduation Rate Schools Dropped


The original data set has 651 schools. 

In [4]:
sy_1819.shape

(134, 277)

# Note on Validation Strategy


This project works with a small sample size. There are only 134 high schools with graduation rates in the dataset. With a larger sample size, I would reserve a holdout set for model validation.  If I were to reserve 20% of the dataset, that would result in less than 30 data points to judge the model.  With such a small set, I would not be confident that the holdout score would be representative of performance on new data.  

Instead, I will use cross-validation, with `k=10`. This will allow for each model to train on over 100 schools. The average score across the 13 or so schools in each holdout will give a good idea of how generalizable the model is across the dataset.  


After the split, there are 27 schools held out in the test set.   I will use a cross val of 4 on the training set to assign a similar number of schools to each fold (~26).

# Modeling on the 4 Main Networks: 14, 15, 16, 17

To begin, I will create a model on the 4 main networks in CPS.  I made this decision because the schools in the other networks in the data set may have different objectives, and graduation rate may be deemphasized as a metric for school success. 

In [9]:
main_networks_1819 = prep_high_school_dataframe('../../data/chicago_data_portal_csv_files/Chicago_Public_Schools_-_School_Profile_Information_SY1819.csv',
                     '../../data/chicago_data_portal_csv_files/Chicago_Public_Schools_-_School_Progress_Reports_SY1819.csv',
                                               isolate_main_nw=True)

0 Student Count
2 schools
15     ENGLEWOOD STEM HS
582       YCCS - VIRTUAL
Name: Short_Name_sp, dtype: object
All 0 Student Count Schools Dropped
0 Graduation Rate
2 schools
261    PATHWAYS - AVONDALE HS
343     NORTHSIDE LEARNING HS
Name: Short_Name_sp, dtype: object
##########
NA Graduation Rates
38 schools
All 0/NA Graduation Rate Schools Dropped


In [13]:
# After isolating the main networks, there are only 68 schools left in the dataset.
main_networks_1819['Network'].shape

(68,)

# FSM: Mean Prediction

## Mean

For a first simple model, I predict the mean of the graduation rates in the training set.

In [6]:
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_validate

y_hat_mean = np.full(sy_1819, np.mean(y_train))

print('R2: Simple prediction of mean')
print(r2_score(y_train,y_hat_mean ))


print('RMSE: Simple prediction of mean')
print(mean_squared_error(y_train,y_hat_mean, squared=False))

R2: Simple prediction of mean
0.0
RMSE: Simple prediction of mean
21.494672805970403


# Linear Regression

## Simple Linear Regression

The FSM only includes one predictor: `Student_Count_Total`, which has a high relative correlation to the target.  

In [8]:
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LinearRegression

lr = LinearRegression()

In [9]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()

y_hat = print_cv_results(lr, X_train, ['Student_Count_Total'], y_train )

#####r2#####
0.20248042617957038
test
[0.1012748  0.1635617  0.09850899]
0.12111516273655187
#####rmse#####
#####Train####
[18.87383651 19.05671021 19.44464238]
19.125063030990777
test
[20.1795537  20.439211   19.09257922]
19.90378130857657
cross val predict metrics: test scores
RMSE
19.91982921575195
r2
0.1411653774043521


## Add perc_low_income_students

In [10]:
lr = LinearRegression()

y_hat = print_cv_results(lr, X_train, ['Student_Count_Total', 'perc_low_income'], y_train )

#####r2#####
0.21790299199412003
test
[0.12840802 0.18193494 0.06837706]
0.126240008169364
#####rmse#####
#####Train####
[18.77693702 18.96632094 19.06150962]
18.934922530708945
test
[19.87260084 20.21348022 19.40903698]
19.83170601003629
cross val predict metrics: test scores
RMSE
19.838378915667313
r2
0.14817440571746188


# Decision Tree Regressor

In [11]:
from sklearn.tree import DecisionTreeRegressor

dt = DecisionTreeRegressor()

y_hat_dt = print_cv_results(dt,X_train, ['Student_Count_Total'], y_train )

#####r2#####
0.9569634244212804
test
[-0.87804874  0.04920533 -0.22270645]
-0.3505166206458689
#####rmse#####
#####Train####
[4.33678176 6.16672694 0.        ]
3.501169566833943
test
[29.17101396 21.79167145 22.23540292]
24.3993627749896
cross val predict metrics: test scores
RMSE
24.653516744076644
r2
-0.3155158352967107


## Add perc_low_income

In [12]:
y_hat = print_cv_results(dt, X_train, ['Student_Count_Total', 'perc_low_income'], y_train )

#####r2#####
1.0
test
[-0.38770439 -0.07178267  0.0250221 ]
-0.14482165384513046
#####rmse#####
#####Train####
[0. 0. 0.]
-0.0
test
[25.07532541 23.13665322 19.855521  ]
22.689166544386143
cross val predict metrics: test scores
RMSE
23.44188080553552
r2
-0.1893871153547504


# Random Forest Regressor

In [15]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(random_state=42)

y_hat_dt = print_cv_results(rf, X_train, ['Student_Count_Total'], y_train )

#####r2#####
0.8351559691111854
test
[-0.44609044  0.28913551  0.0609022 ]
-0.03201757499461866
#####rmse#####
#####Train####
[ 6.97731502 10.22706615  8.41695865]
8.540446606361657
test
[25.59739812 18.84258936 19.48674544]
21.30891097083011
cross val predict metrics: test scores
RMSE
21.543345217399533
r2
-0.0045339157874995095


## Add perc_low_income

In [16]:
y_hat = print_cv_results(rf, X_train, ['Student_Count_Total', 'perc_low_income'], y_train )

#####r2#####
0.8755205192683565
test
[0.17312212 0.35463075 0.12792682]
0.2185598948217468
#####rmse#####
#####Train####
[6.08589696 8.24089836 8.10699009]
7.4779284717728105
test
[19.35614132 17.95359016 18.77847701]
18.696069494374026
cross val predict metrics: test scores
RMSE
18.704237558001886
r2
0.24278651047274669


# Best so far: 18.704 RMSE

# Inverse Log transform target

In [18]:
y_hat_train = print_cv_results(lr,X_train, ['Student_Count_Total'], np.log(100-y_train) )

#####r2#####
0.24332259579712523
test
[0.1310171  0.21155901 0.11198908]
0.1515217300353819
#####rmse#####
#####Train####
[0.67627829 0.72975271 0.6978114 ]
0.7012808018773812
test
[0.77365276 0.70009198 0.73488376]
0.7362095011457256
cross val predict metrics: test scores
RMSE
0.7368404420480538
r2
0.17029449357156934


In [19]:
from sklearn.metrics import mean_squared_error, r2_score
y_hat_train_untransformed = 100 - np.e**y_hat_train

rmse = mean_squared_error(y_train, y_hat_train_untransformed, squared=False)
print(rmse)

r2 = r2_score(y_train, y_hat_train_untransformed)
print(r2)

20.156315026940735
0.12065236972536697


## One Hot Encode Networks

In [ ]:
from sklearn.impute import SimpleImputer

network_imputer = SimpleImputer(strategy='constant', fill_value='missing')

X_train['Network'] = network_imputer.fit_transform(X_train[['Network']])

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder


# network_imputer = ColumnTransformer([('network_impute', SimpleImputer(strategy='constant', fill_value='missing'), ['Network'])], 
#                                          remainder='passthrough')


# network_transformer = ColumnTransformer( [('network_impute', SimpleImputer(strategy='constant', fill_value='missing'), ['Network']), 
#                                          ('network_ohe', OneHotEncoder(handle_unknown='ignore'), ['Network'])], 
#                                          remainder='passthrough')


network_ohe = ColumnTransformer([ 
                                          ('network_ohe', OneHotEncoder(handle_unknown='ignore'), ['Network'])], 
                                         remainder='passthrough')

network_pipe = make_pipeline(network_ohe, LinearRegression())

cv_cps(network_pipe, X_train, y_train, ['Student_Count_Total', 'perc_low_income', 'Network'])

Adding Network One Hot Encoded columns decreases the bias, but increases the vaiance.  There is even one negative R^2 score which is particularly worrisome.

# High Correlation 

In [ ]:
df_train.corr().iloc[-1].dropna().sort_values()

In [ ]:
df_train.corr().iloc[-1].dropna().sort_values(ascending=False)

In [ ]:
df_train[['Graduation_Rate_School', 'Student_Count_Low_Income']].corr()